In [2]:
from langchain_ollama import ChatOllama

llm = ChatOllama(model="llama3.2:1b")


In [3]:
from langchain_core.prompts import PromptTemplate
from langchain_core.output_parsers import StrOutputParser

# 명시적인 지시사항이 포함된 프롬프트 템플릿 정의
prompt_template = PromptTemplate(
    template="What is the capital of {country}? Return the name of the city only",
    input_variables=["country"],
)
# 템플릿에 변수 값을 주입
prompt = prompt_template.invoke({"country": "France"})

print(prompt)

# LLM에 프롬프트 전달
ai_message = llm.invoke(prompt_template.invoke({"country": "France"}))

print(ai_message)

# 문자열 출력 파서 설정
output_parser = StrOutputParser()

answer = output_parser.invoke(llm.invoke(prompt_template.invoke({"country": "France"})))

text='What is the capital of France? Return the name of the city only'
content='Paris' additional_kwargs={} response_metadata={'model': 'llama3.2:1b', 'created_at': '2025-03-25T06:44:13.9659621Z', 'done': True, 'done_reason': 'stop', 'total_duration': 2102967900, 'load_duration': 1678795600, 'prompt_eval_count': 39, 'prompt_eval_duration': 370187800, 'eval_count': 2, 'eval_duration': 53478400, 'message': Message(role='assistant', content='', images=None, tool_calls=None)} id='run-52b09011-61e9-4062-8f76-69f98b9af000-0' usage_metadata={'input_tokens': 39, 'output_tokens': 2, 'total_tokens': 41}


In [5]:
capital_chain = prompt_template | llm | output_parser

In [6]:
capital_chain.invoke({"country": "France"})

'Paris.'

### 복잡한 체인 구성

In [8]:
# 국가를 추측하는 프롬프트 템플릿 정의
country_prompt = PromptTemplate(
    template="""Guess the name of the country in the {continent} based on the following information:
    {information}
    Return the name of the country only
    """,
    input_variables=["information", "continent"],
)

# 국가 추측 체인 생성
country_chain = country_prompt | llm | output_parser
# country_chain.invoke({"information": "This country is very famous for its wine in Europe"})

In [9]:
from langchain_core.runnables import RunnablePassthrough

# RunnablePassthrough를 사용하여 입력을 다음 단계로 전달하는 복합 체인 구성
final_chain = {"information": RunnablePassthrough(), "continent": RunnablePassthrough()} | {"country": country_chain} | capital_chain

In [10]:
# 복합 체인 실행
# 정보와 대륙을 입력하면, 해당 국가를 추측하고 그 국가의 수도를 반환
final_chain.invoke( {"information": "This country is very famous for its wine in Europe", "continent": "Europe"})

'Rome.'